## One class classification

In [48]:
import librosa
import math
from scipy.io import wavfile

def precompute_min_timeseries_len_librosa(list_of_audiofiles, hop_length, sr):
    timeseries_length_list = []
    for file in list_of_audiofiles:
        #print("Loading " + str(file))
        y, sr = librosa.load(file, sr=sr)
        timeseries_length_list.append(math.ceil(len(y) / hop_length))
    return min(timeseries_length_list)

def precompute_min_timeseries_len_wavfile(list_of_audiofiles, hop_length):
    timeseries_length_list = []
    for file in list_of_audiofiles:
        #print("Loading " + str(file))
        sr, y = wavfile.read(file)
        timeseries_length_list.append(math.ceil(len(y) / hop_length))
    return min(timeseries_length_list)

def precompute_min_timeseries_len_from_data(array_of_data, hop_length):
    timeseries_length_list = []
    for data in array_of_data:
        timeseries_length_list.append(math.ceil(len(data) / hop_length))
    return min(timeseries_length_list)

def extract_audio_features_librosa(list_of_audiofiles, sound_class, hop_length, sr):
    list_of_data = []
    for i, file in enumerate(list_of_audiofiles):
        y, sr = librosa.load(file, sr=sr)
        list_of_data.append(y)
    return extract_audio_features_from_data(list_of_data, sound_class, hop_length, sr)

def extract_audio_features_wavfile(list_of_audiofiles, sound_class, hop_length):
    list_of_data = []
    for i, file in enumerate(list_of_audiofiles):
        print(file)
        sr, y = wavfile.read(file)
        list_of_data.append(y)
    return extract_audio_features_from_data(list_of_data, sound_class, hop_length, sr)

def extract_audio_features_from_data(list_of_data, sound_class, hop_length, sr):
    n_mfcc = 13
    whole_data = None
    
    for i, y in enumerate(list_of_data):
        y = y.astype(np.float32)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=n_mfcc)
        spectral_center = librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=hop_length)
        chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length)
        spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length)
    
        timeseries_length = mfcc.T.shape[0] # precompute_min_timeseries_len_from_data([y], hop_length) + 1
        data = np.zeros((timeseries_length, 33), dtype=np.float64)
        #print('data shape: ', data.shape)
        data[:, 0:13] = mfcc.T
        data[:, 13:14] = spectral_center.T
        data[:, 14:26] = chroma.T
        data[:, 26:33] = spectral_contrast.T
        
        #print('timeseries_length: ', timeseries_length)
        #print('sound_data: ', y[:20])
        #print('shape: ', data.shape)
        #print('before norm: ', data[:20, :])
        data = data / np.linalg.norm(data)
        #print('after norm: ', data[:20, :])
        
        if whole_data is None:
            whole_data = data
        else:
            whole_data = np.vstack((whole_data, data))
        
    return whole_data, np.full((whole_data.shape[0], 1), sound_class)

In [49]:
def print_metrics(data):
        familiar_count = float(len(data[data == 1]))
        print('familiar_count', familiar_count)
        all_data_count = float(len(data))
        print('all_data_count', all_data_count)
        familiar_percent = familiar_count / all_data_count
        print('familiar_percent', familiar_percent)
        print()

In [50]:
timeseries_length_list = []
hop_length = 512
for sound_class, sound_paths in class_to_file_paths.items():
    timeseries_length_list.append(precompute_min_timeseries_len_wavfile(sound_paths, hop_length))
print(timeseries_length_list)

[46, 133, 110, 139]


/usr/local/anaconda3/lib/python3.6/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [51]:
import numpy as np

hop_length = 512
X_train_one_svm = None
Y_train_one_svm = None

for sound_class, sound_paths in class_to_file_paths.items():
    data, target = extract_audio_features_wavfile(sound_paths, int(sound_class), hop_length)
    if X_train_one_svm is None and Y_train_one_svm is None:
        X_train_one_svm = data
        Y_train_one_svm = target
    else:
        X_train_one_svm = np.vstack((X_train_one_svm, data))
        Y_train_one_svm = np.vstack((Y_train_one_svm, target))
print(X_train_one_svm.shape, Y_train_one_svm.shape)

sounds-with-classes-without-noise/1/stepan-1.wav
sounds-with-classes-without-noise/1/stepan-2.wav
sounds-with-classes-without-noise/1/stepan-3.wav
sounds-with-classes-without-noise/1/stepan-7.wav
sounds-with-classes-without-noise/1/stepan-6.wav
sounds-with-classes-without-noise/1/stepan-4.wav
sounds-with-classes-without-noise/1/stepan-5.wav
sounds-with-classes-without-noise/1/stepan-24.wav
sounds-with-classes-without-noise/1/stepan-18.wav
sounds-with-classes-without-noise/1/stepan-19.wav
sounds-with-classes-without-noise/1/stepan-22.wav
sounds-with-classes-without-noise/1/stepan-23.wav
sounds-with-classes-without-noise/1/stepan-21.wav
sounds-with-classes-without-noise/1/stepan-20.wav
sounds-with-classes-without-noise/1/stepan-11.wav
sounds-with-classes-without-noise/1/stepan-10.wav
sounds-with-classes-without-noise/1/stepan-12.wav
sounds-with-classes-without-noise/1/stepan-13.wav
sounds-with-classes-without-noise/1/stepan-17.wav
sounds-with-classes-without-noise/1/stepan-16.wav
sounds-

In [71]:
from sklearn import svm

clf = svm.OneClassSVM(nu=0.1, coef0=0.09, kernel="poly", gamma=0.1)
clf.fit(X_train_one_svm)
print_metrics(clf.predict(X_train_one_svm))

familiar_count 7939.0
all_data_count 8711.0
familiar_percent 0.9113764206176099



In [72]:
import pickle
pickle.dump(clf, open('one_class_model_for_5_classes.pkl', 'wb'))

In [ ]:
fake_files_dir='fake-sounds'
fake_files_paths = get_sound_files_paths(fake_files_dir)
fake_data = read_full_data_from_drive(fake_files_paths)

In [ ]:
print(fake_data)

In [ ]:
for sound_class, sound_paths in fake_data.items():
    for sound_path in sound_paths:
        data, target = extract_audio_features_wavfile([sound_path], int(sound_class), hop_length)
        print_metrics(clf.predict(data))

In [ ]:
for sound_class, sound_paths in class_to_file_paths.items():
    for sound_path in sound_paths:
        data, target = extract_audio_features_wavfile([sound_path], int(sound_class), hop_length)
        print_metrics(clf.predict(data))

## LSTM model

In [30]:
#sound_files_dir='sounds-with-classes'
sound_files_dir='sounds-with-classes-without-noise'

In [31]:
def get_sound_files_paths(sound_files_dir):
    from os import walk
    sound_files_paths = []
    for (dirpath, dirnames, filenames) in walk(sound_files_dir):
        if len(filenames) > 0:
            for filename in filenames:
                if len(filename) > 0 and filename[0] != '.':
                    sound_files_paths.append('/'.join([dirpath, filename]))
    return sound_files_paths

In [32]:
sound_files_paths = get_sound_files_paths(sound_files_dir)

In [33]:
def read_full_data_from_drive(data_path):
    class_to_file_paths = {}
    for sound_file_path in data_path:
        file_class = sound_file_path.split('/')[-2]
        if file_class in class_to_file_paths:
            class_to_file_paths[file_class].append(sound_file_path)
        else:
            class_to_file_paths[file_class] = [sound_file_path]
    return class_to_file_paths

def read_data_from_drive(sound_file_path):
    from scipy.io import wavfile
    fs, data = wavfile.read(sound_file_path)
    #print(sound_file_path)
    #print(data[:20])
    #print(data.dtype)
    return data

In [34]:
class_to_file_paths = read_full_data_from_drive(sound_files_paths)

In [35]:
print(class_to_file_paths)

{'1': ['sounds-with-classes-without-noise/1/stepan-1.wav', 'sounds-with-classes-without-noise/1/stepan-2.wav', 'sounds-with-classes-without-noise/1/stepan-3.wav', 'sounds-with-classes-without-noise/1/stepan-7.wav', 'sounds-with-classes-without-noise/1/stepan-6.wav', 'sounds-with-classes-without-noise/1/stepan-4.wav', 'sounds-with-classes-without-noise/1/stepan-5.wav', 'sounds-with-classes-without-noise/1/stepan-24.wav', 'sounds-with-classes-without-noise/1/stepan-18.wav', 'sounds-with-classes-without-noise/1/stepan-19.wav', 'sounds-with-classes-without-noise/1/stepan-22.wav', 'sounds-with-classes-without-noise/1/stepan-23.wav', 'sounds-with-classes-without-noise/1/stepan-21.wav', 'sounds-with-classes-without-noise/1/stepan-20.wav', 'sounds-with-classes-without-noise/1/stepan-11.wav', 'sounds-with-classes-without-noise/1/stepan-10.wav', 'sounds-with-classes-without-noise/1/stepan-12.wav', 'sounds-with-classes-without-noise/1/stepan-13.wav', 'sounds-with-classes-without-noise/1/stepan-17

In [36]:
import IPython.display as ipd
ipd.Audio(read_data_from_drive(class_to_file_paths['4'][0]), rate=44100)

In [37]:
def split_data_to_train_and_valid(data, valid_percent=20.0, test_percent=10.0):
    import math
    import random
    train_data = {}
    valid_data = {}
    for class_id, sound_files_paths in data.items():
        valid_samples_in_class = math.floor(len(sound_files_paths) * (valid_percent / 100))
        train_samples_in_class = len(sound_files_paths) - valid_samples_in_class
        valid_samples_ids = random.sample(range(len(sound_files_paths)), valid_samples_in_class)
        for i in range(len(sound_files_paths)):
            if i in valid_samples_ids:
                valid_data[sound_files_paths[i]] = class_id
            else:
                train_data[sound_files_paths[i]] = class_id
    return (train_data, valid_data)

In [38]:
train_data, valid_data = split_data_to_train_and_valid(class_to_file_paths)

In [39]:
print(valid_data)

{'sounds-with-classes-without-noise/1/stepan-7.wav': '1', 'sounds-with-classes-without-noise/1/stepan-6.wav': '1', 'sounds-with-classes-without-noise/1/stepan-16.wav': '1', 'sounds-with-classes-without-noise/1/stepan-15.wav': '1', 'sounds-with-classes-without-noise/4/item-api-13.wav': '4', 'sounds-with-classes-without-noise/4/item-api-7.wav': '4', 'sounds-with-classes-without-noise/3/kuber-show-4.wav': '3', 'sounds-with-classes-without-noise/3/kuber-show-6.wav': '3', 'sounds-with-classes-without-noise/3/kuber-show-11.wav': '3', 'sounds-with-classes-without-noise/2/catalog-api-show-8.wav': '2', 'sounds-with-classes-without-noise/2/catalog-api-show-10.wav': '2', 'sounds-with-classes-without-noise/2/catalog-api-show-4.wav': '2'}


In [40]:
def get_spectrogram_of_file(samples, sample_rate=44100):
    from scipy import signal
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    return spectrogram.T

In [41]:
def get_items_count(data):
    items_count = 0
    for file_path in data:
        items_count += len(read_data_from_drive(file_path))
    return items_count

def get_spectrogram_count(data):
    items_count = 0
    for file_path in data:
        items_count += get_spectrogram_of_file(read_data_from_drive(file_path)).shape[0]
    return items_count

In [42]:
train_data_items_count = get_spectrogram_count(train_data)
valid_data_items_count = get_spectrogram_count(valid_data)

In [43]:
print(train_data)
print(valid_data)

{'sounds-with-classes-without-noise/1/stepan-1.wav': '1', 'sounds-with-classes-without-noise/1/stepan-2.wav': '1', 'sounds-with-classes-without-noise/1/stepan-3.wav': '1', 'sounds-with-classes-without-noise/1/stepan-4.wav': '1', 'sounds-with-classes-without-noise/1/stepan-5.wav': '1', 'sounds-with-classes-without-noise/1/stepan-24.wav': '1', 'sounds-with-classes-without-noise/1/stepan-18.wav': '1', 'sounds-with-classes-without-noise/1/stepan-19.wav': '1', 'sounds-with-classes-without-noise/1/stepan-22.wav': '1', 'sounds-with-classes-without-noise/1/stepan-23.wav': '1', 'sounds-with-classes-without-noise/1/stepan-21.wav': '1', 'sounds-with-classes-without-noise/1/stepan-20.wav': '1', 'sounds-with-classes-without-noise/1/stepan-11.wav': '1', 'sounds-with-classes-without-noise/1/stepan-10.wav': '1', 'sounds-with-classes-without-noise/1/stepan-12.wav': '1', 'sounds-with-classes-without-noise/1/stepan-13.wav': '1', 'sounds-with-classes-without-noise/1/stepan-17.wav': '1', 'sounds-with-class

In [44]:
def build_vocabulary(class_to_file_paths):
    vocabulary = {}
    vocab_index = 0
    for file_class, file_paths in class_to_file_paths.items():
        for file_path in file_paths:
            file_data = read_data_from_drive(file_path)
            for file_item in file_data:
                if file_item not in vocabulary:
                    vocabulary[file_item] = vocab_index
                    vocab_index += 1
    reversed_vocabulary = dict(zip(vocabulary.values(), vocabulary.keys()))
    return (vocabulary, reversed_vocabulary)

In [45]:
vocabulary, reversed_vocabulary = build_vocabulary(class_to_file_paths)

In [46]:
from keras.utils import to_categorical
import numpy as np

class StepanBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, num_classes, vocabulary, skip_steps=1, freq=129):
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.skip_steps = skip_steps
        self.vocabulary = vocabulary
        self.freq = freq
        
        self.current_data = None
        self.current_file_idx = 0
        self.current_pos_in_file = 0
        
        self.current_spectrogram = None
        self.current_pos_in_spectrogram = 0

        self.file_paths = []
        self.file_classes = []
        for file_path, file_class in data.items():
            self.file_paths.append(file_path)
            self.file_classes.append(file_class)
        self.__change_file()
    
    def __map_data_to_vocabulary(self, data):
        return [self.vocabulary[d] for d in data if d in self.vocabulary]
    
    def __change_file(self):
        while True:
            try:
                if self.current_file_idx+1 >= len(self.file_paths):
                    self.current_file_idx = 0
                else:
                    self.current_file_idx += 1
                self.current_data = read_data_from_drive(self.file_paths[self.current_file_idx])
                self.current_spectrogram = get_spectrogram_of_file(self.current_data)
                self.current_pos_in_file = 0
                self.current_pos_in_spectrogram = 0
                break
            except:
                pass
        
    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps, self.freq))
        y = np.zeros((self.batch_size, self.num_classes))
        while True:
            for i in range(self.batch_size):
                if self.current_pos_in_spectrogram + self.num_steps >= len(self.current_spectrogram):
                    self.__change_file()
                x[i, :, :] = self.current_spectrogram[self.current_pos_in_spectrogram:self.current_pos_in_spectrogram + self.num_steps, :]
                y[i, :] = to_categorical(self.file_classes[self.current_file_idx], num_classes=self.num_classes)
                self.current_pos_in_spectrogram += self.skip_steps
            yield x, y

In [47]:
num_steps = 60
skip_steps = 30
batch_size = 16
num_classes = len(class_to_file_paths)

In [19]:
train_data_generator = StepanBatchGenerator(train_data, num_steps, batch_size, num_classes, vocabulary, skip_steps=skip_steps)
valid_data_generator = StepanBatchGenerator(valid_data, num_steps, batch_size, num_classes, vocabulary, skip_steps=skip_steps)

In [20]:
def pieces_in_whole_sound_with_step(data_count, num_steps, skip_steps):
    pieces = 0
    end_piece = num_steps
    while end_piece <= data_count:
        pieces += 1
        end_piece += skip_steps
    return pieces

In [21]:
train_steps = pieces_in_whole_sound_with_step(train_data_items_count, num_steps, skip_steps)//batch_size
validation_steps = pieces_in_whole_sound_with_step(valid_data_items_count, num_steps, skip_steps)//batch_size
print('Train steps: ', train_steps)
print('Valid steps: ', validation_steps)

Train steps:  53
Valid steps:  12


In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, LSTM
from keras.initializers import Constant, RandomNormal

hidden_size = 600
num_epochs = 30

stepan_model = Sequential()
stepan_model.add(LSTM(hidden_size, dropout=0.0, return_sequences=False, input_shape=(num_steps, 129)))
stepan_model.add(Dense(units=num_classes, activation='softmax'))
                 
stepan_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
print(stepan_model.summary())
                 
hist = stepan_model.fit_generator(train_data_generator.generate(), train_steps, num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=validation_steps)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 600)               1752000   
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 3005      
Total params: 1,755,005
Trainable params: 1,755,005
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
53/53 [==============================] - 17s 312ms/step - loss: 3.3486 - categorical_accuracy: 0.3255 - val_loss: 1.2773 - val_categorical_accuracy: 0.5938
Epoch 2/30
53/53 [==============================] - 17s 318ms/step - loss: 1.7717 - categorical_accuracy: 0.4045 - val_loss: 1.0709 - val_categorical_accuracy: 0.6406
Epoch 3/30
53/53 [==============================] - 18s 349ms/step - loss: 1.4160 - categorical_accuracy: 0.4446 - val_loss: 0.8889 - val_categorical_accuracy: 0.6354
Epoch 4/30
53/53 [====

In [27]:
stepan_model.save("stepan_model_5_classes_(noise,stepan,catalog,kuber,item)_acc_84.hdf5")

In [28]:
from keras.models import load_model
stepan_model = load_model("stepan_model_5_classes_(noise,stepan,catalog,kuber,item)_acc_84.hdf5")

In [ ]:
fake_files_dir='fake-sounds'
fake_files_paths = get_sound_files_paths(fake_files_dir)
fake_data = read_full_data_from_drive(fake_files_paths)
fake_data = split_data_to_train_and_valid(fake_data, valid_percent=0.0)
fake_data = fake_data[0]
for path_to_file, cl in fake_data.items():
    fake_data[path_to_file] = '0'

In [ ]:
dummy_iters = 0
test_data = fake_data
example_training_generator = StepanBatchGenerator(test_data, num_steps, 1, num_classes, vocabulary, skip_steps=skip_steps)
print("Training data:")
#for i in range(dummy_iters):
#    dummy = next(example_training_generator.generate())
num_predict = 300
for i in range(num_predict):
    data = next(example_training_generator.generate())
    prediction = stepan_model.predict_proba(data[0])
    predicted = np.argmax(prediction)
    true = np.argmax(data[1])
    print('Label {}: predicted: {} true: {} p: {}'.format(i, predicted, true, prediction))